# WeRateDogs

## Introduction

@dog_rates WeRateDogs
The ratings that they have archived from Twitter have been filtered from 5000 to 2356 tweets. These tweets have ratings. However, some details such as the ratings, dog names, dog stages are not correct. Dog stages are:  doggo, pupper, puppo, and floof(er).
Going to use the Twitter Archive to find the retweet and favourite count. 
Prediction tweets.
Want original images (no retweets). In the dataset, not all of them are original.
Assess and clean at least 8 quality issues and at least 2 tidiness issues.
Do not gather tweets beyond August 1st 2017, won't be able to gather image preidictions passed this date.


## To Do

- Get a Twitter Developer account
- Get the Twitter API keys
- Download the Twitter archive given by Udacity
- Download the Tweet images do this by using the Requests library 
- Identify at least 8 quality and 2 tidiness issues
- Assess and clean these issues
- Correct the ratings, dog names, dog stages
- Gather original images, not retweets
- Gather the retweet and favourite count
- Install the Tweepy library


## Import libraries 

In [1]:
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
import requests
import json
import os
from timeit import default_timer as timer

## Gathering Datasets:
Gathering Twitter archive given by Udacity, images from Tweets, and retweet/favourite count from Twitter's API. 

### 1. Twitter Archive Data

In [2]:
df_arc = pd.read_csv("twitter-archive-enhanced.csv")
df_arc.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
#drop timestamp, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls
arc_clean = df_arc.copy()
arc_clean.drop(columns = ["timestamp","in_reply_to_status_id","in_reply_to_user_id","retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp","expanded_urls"], axis = 1, inplace = True)
arc_clean.head()

,tweet_id,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None,None
1,892177421306343426,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,None,None,None,None
2,891815181378084864,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,None,None,None,None
3,891689557279858688,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13,10,Darla,None,None,None,None
4,891327558926688256,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12,10,Franklin,None,None,None,None


In [4]:
arc_clean.duplicated().sum()

0

In [5]:
#Some url's do not work because the column has been truncated in the dataframe
pd.set_option('display.max_colwidth', -1)
arc_clean

,tweet_id,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,13,10,Phineas,None,None,None,None
1,892177421306343426,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",13,10,Tilly,None,None,None,None
2,891815181378084864,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,12,10,Archie,None,None,None,None
3,891689557279858688,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,13,10,Darla,None,None,None,None
4,891327558926688256,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",12,10,Franklin,None,None,None,None
5,891087950875897856,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,13,10,None,None,None,None,None
6,890971913173991426,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,13,10,Jax,None,None,None,None
7,890729181411237888,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,13,10,None,None,None,None,None
8,890609185150312448,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,13,10,Zoey,None,None,None,None
9,890240255349198849,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,14,10,Cassie,doggo,None,None,None


In [6]:
#change the timestamp to datetime instead of a object datatype


In [7]:
#From the text column, seperate the the text, ratings, and url. Put these into different columns.
arc_clean["texts"], arc_clean["url"] = arc_clean["text"].str.rsplit(" ", 1).str
arc_clean.drop(columns = ["text"], axis = 1, inplace = True)
arc_clean

,tweet_id,source,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,texts,url
0,892420643555336193,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13,10,Phineas,None,None,None,None,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10,https://t.co/MgUWQ76dJU
1,892177421306343426,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13,10,Tilly,None,None,None,None,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10",https://t.co/0Xxu71qeIV
2,891815181378084864,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",12,10,Archie,None,None,None,None,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10,https://t.co/wUnZnhtVJB
3,891689557279858688,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13,10,Darla,None,None,None,None,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us,https://t.co/tD36da7qLQ
4,891327558926688256,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",12,10,Franklin,None,None,None,None,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek",https://t.co/AtUZn91f7f
5,891087950875897856,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13,10,None,None,None,None,None,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek,https://t.co/kQ04fDDRmh
6,890971913173991426,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13,10,Jax,None,None,None,None,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H,https://t.co/tVJBRMnhxl
7,890729181411237888,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13,10,None,None,None,None,None,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10,https://t.co/v0nONBcwxq
8,890609185150312448,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13,10,Zoey,None,None,None,None,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek,https://t.co/9TwLuAGH0b
9,890240255349198849,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",14,10,Cassie,doggo,None,None,None,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate,https://t.co/t1bfwz5S2A


In [8]:
#source? Are they all from iphones? One is from web client one is from vine.
arc_clean.sample(5)
arc_clean["link"], arc_clean["sourced"] = arc_clean["source"].str.split(">", 1).str
arc_clean["sourced"] = arc_clean["sourced"].str.replace("</a>", " ")
arc_clean.drop(columns = ["link","source"], axis = 1, inplace = True)
arc_clean

,tweet_id,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,texts,url,sourced
0,892420643555336193,13,10,Phineas,None,None,None,None,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10,https://t.co/MgUWQ76dJU,Twitter for iPhone
1,892177421306343426,13,10,Tilly,None,None,None,None,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10",https://t.co/0Xxu71qeIV,Twitter for iPhone
2,891815181378084864,12,10,Archie,None,None,None,None,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10,https://t.co/wUnZnhtVJB,Twitter for iPhone
3,891689557279858688,13,10,Darla,None,None,None,None,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us,https://t.co/tD36da7qLQ,Twitter for iPhone
4,891327558926688256,12,10,Franklin,None,None,None,None,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek",https://t.co/AtUZn91f7f,Twitter for iPhone
5,891087950875897856,13,10,None,None,None,None,None,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek,https://t.co/kQ04fDDRmh,Twitter for iPhone
6,890971913173991426,13,10,Jax,None,None,None,None,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H,https://t.co/tVJBRMnhxl,Twitter for iPhone
7,890729181411237888,13,10,None,None,None,None,None,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10,https://t.co/v0nONBcwxq,Twitter for iPhone
8,890609185150312448,13,10,Zoey,None,None,None,None,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek,https://t.co/9TwLuAGH0b,Twitter for iPhone
9,890240255349198849,14,10,Cassie,doggo,None,None,None,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate,https://t.co/t1bfwz5S2A,Twitter for iPhone


In [9]:
arc_clean["name"].unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', 'None', 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver', 'Jim',
       'Zeke', 'Ralphus', 'Canela', 'Gerald', 'Jeffrey', 'such', 'Maya',
       'Mingus', 'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey', 'Lilly',
       'Earl', 'Lola', 'Kevin', 'Yogi', 'Noah', 'Bella', 'Grizzwald',
       'Rusty', 'Gus', 'Stanley', 'Alfy', 'Koko', 'Rey', 'Gary', 'a',
       'Elliot', 'Louis', 'Jesse', 'Romeo', 'Bailey', 'Duddles', 'Jack',
       'Emmy', 'Steven', 'Beau', 'Snoopy', 'Shadow', 'Terrance', 'Aja',
       'Penny', 'Dante', 'Nelly', 'Ginger', 'Benedict', 'Venti', 'Goose',
       'Nugget', 'Cash', 'Coco', 'Jed', 'Sebastian', 'Walter', 'Sierra',
       'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover', 'Napolean', 'Dawn',
       'Boomer', 'Cody', 'Rumble', 'Clifford', 'quite', 'Dewey', 'Scout',
       'Gizmo', 'Cooper', 'Harold', 'Shikha', 'Jamesy', 'Lili', 'Sammy',
       'Meatball', 'Paisley', 'Albus', 'Neptune', '

In [10]:
#the names are in capital letters, gather the unique lowercase words
no_name = set()

for word in arc_clean["name"]:
    if word.islower():
        no_name.add(word)
print(no_name)

{'quite', 'actually', 'this', 'just', 'the', 'mad', 'life', 'one', 'old', 'my', 'unacceptable', 'infuriating', 'space', 'light', 'all', 'a', 'his', 'officially', 'an', 'not', 'by', 'such', 'very', 'incredibly', 'getting'}


In [11]:
#replace the words that are not names with "None"
arc_clean = arc_clean.replace({'name': ['incredibly', 'his', 'life', 
                   'unacceptable', 'not', 'such', 'by', 
                   'quite', 'light', 'my', 'old', 'space', 
                   'infuriating', 'the', 'a', 'all', 'actually', 
                   'officially', 'getting', 'just', 'mad', 'one', 
                   'very', 'this', 'an']}, "None")

In [12]:
#test if "his" is in the name column
arc_clean[arc_clean["name"]=="his"].count()

tweet_id              0
rating_numerator      0
rating_denominator    0
name                  0
doggo                 0
floofer               0
pupper                0
puppo                 0
texts                 0
url                   0
sourced               0
dtype: int64

In [13]:
#Some of the names are written in the text column
#758 is Name = Loki, 1684 = Toby, 775 = O'Malley
arc_clean.iloc[758]

tweet_id              778408200802557953                                                       
rating_numerator      14                                                                       
rating_denominator    10                                                                       
name                  None                                                                     
doggo                 None                                                                     
floofer               None                                                                     
pupper                None                                                                     
puppo                 None                                                                     
texts                 RIP Loki. Thank you for the good times. You will be missed by many. 14/10
url                   https://t.co/gJKD9pst5A                                                  
sourced               Twitter for iPhone

In [14]:
#rating demoninator always has to equal to 10. 
arc_clean["rating_denominator"] = 10
arc_clean["rating_denominator"].sum() #23560
arc_clean.info()

#rating_denominator has 2356 entries, and the sum of rating_denominator = 23560. Therefore, each rating_denominator = 10

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2356 non-null int64
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
name                  2356 non-null object
doggo                 2356 non-null object
floofer               2356 non-null object
pupper                2356 non-null object
puppo                 2356 non-null object
texts                 2356 non-null object
url                   2355 non-null object
sourced               2356 non-null object
dtypes: int64(3), object(8)
memory usage: 202.5+ KB


In [15]:
arc_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2356 non-null int64
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
name                  2356 non-null object
doggo                 2356 non-null object
floofer               2356 non-null object
pupper                2356 non-null object
puppo                 2356 non-null object
texts                 2356 non-null object
url                   2355 non-null object
sourced               2356 non-null object
dtypes: int64(3), object(8)
memory usage: 202.5+ KB


In [16]:
#doggo, floofer, pupper, puppo has to be in one column called dog stages. 
arc_clean = arc_clean.melt(id_vars = ["name", "texts","tweet_id","rating_numerator", "rating_denominator", "url", "sourced"], 
                               var_name = "dogs", value_name = "dog_stage")

arc_clean

#, value_name = "" value_vars = ["doggo", "floofer", "pupper", "puppo"]

,name,texts,tweet_id,rating_numerator,rating_denominator,url,sourced,dogs,dog_stage
0,Phineas,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10,892420643555336193,13,10,https://t.co/MgUWQ76dJU,Twitter for iPhone,doggo,None
1,Tilly,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10",892177421306343426,13,10,https://t.co/0Xxu71qeIV,Twitter for iPhone,doggo,None
2,Archie,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10,891815181378084864,12,10,https://t.co/wUnZnhtVJB,Twitter for iPhone,doggo,None
3,Darla,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us,891689557279858688,13,10,https://t.co/tD36da7qLQ,Twitter for iPhone,doggo,None
4,Franklin,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek",891327558926688256,12,10,https://t.co/AtUZn91f7f,Twitter for iPhone,doggo,None
5,None,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek,891087950875897856,13,10,https://t.co/kQ04fDDRmh,Twitter for iPhone,doggo,None
6,Jax,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H,890971913173991426,13,10,https://t.co/tVJBRMnhxl,Twitter for iPhone,doggo,None
7,None,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10,890729181411237888,13,10,https://t.co/v0nONBcwxq,Twitter for iPhone,doggo,None
8,Zoey,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek,890609185150312448,13,10,https://t.co/9TwLuAGH0b,Twitter for iPhone,doggo,None
9,Cassie,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate,890240255349198849,14,10,https://t.co/t1bfwz5S2A,Twitter for iPhone,doggo,doggo


In [17]:
#this shows that the melt function makes 4 of each, where dog_stage will be doggo, floofer, pupper,  puppo.
#the value is the one you want 
arc_clean.query('name == "Roscoe"')

#drop the dogs column
arc_clean = arc_clean.drop(["dogs"], axis =1)


In [18]:
#The melt function has given us duplicates
arc_clean = arc_clean.drop_duplicates()
arc_clean.texts.duplicated().shape
#No because it'll remove some dogs that have not got a defined stage

(2750,)

In [19]:
#keep last in all duplicates in text apart from the doggo stages
#to do this, the doggo in their dog stage are placed into a new df

doggos = arc_clean.query('dog_stage == "doggo"')
doggos.shape

(97, 8)

In [20]:
#keep last in duplicated texts

arc_clean = arc_clean.drop_duplicates(subset=['texts'], keep='last')
arc_clean.shape

(2356, 8)

In [21]:
#merge the doggos with the rest of the dogs (arc_clean)

arc_clean = pd.concat([arc_clean, doggos])
arc_clean.shape

(2453, 8)

In [22]:
arc_clean.loc[150]

name                  Quinn                                                                                                                                        
texts                 Say hello to Quinn. She's quite the goofball. Not even a year old. Confirmed 13/10 but she really needs your help \n\nhttps://t.co/MOBkQnyHib
tweet_id              863062471531167744                                                                                                                           
rating_numerator      13                                                                                                                                           
rating_denominator    10                                                                                                                                           
url                   https://t.co/EsOB4rLEKt                                                                                                                      
sourced         

In [23]:
arc_clean.query('dog_stage == "floofer"')

,name,texts,tweet_id,rating_numerator,rating_denominator,url,sourced,dog_stage
2402,Grizzwald,Meet Grizzwald. He may be the floofiest floofer I ever did see. Lost eyes saving a schoolbus from a volcano erpuption. 13/10 heroic as h*ck,883360690899218434,13,10,https://t.co/rf661IFEYP,Twitter for iPhone,floofer
2938,Doc,This is Doc. He takes time out of every day to worship our plant overlords. 12/10 quite the floofer,800388270626521089,12,10,https://t.co/azMneS6Ly5,Twitter for iPhone,floofer
3130,None,Atlas rolled around in some chalk and now he's a magical rainbow floofer. 13/10 please never take a bath,776218204058357768,13,10,https://t.co/nzqTNw0744,Twitter for iPhone,floofer
3340,Blu,This is Blu. He's a wild bush Floofer. I wish anything made me as happy as bushes make Blu. 12/10 would frolic with,749317047558017024,12,10,https://t.co/HHUAnBb6QB,Twitter for iPhone,floofer
3378,None,Here's a golden floofer helping with the groceries. Bed got in way. Still 11/10 helpful af (vid by @categoen),746542875601690625,11,10,https://t.co/6ZRoZUWFmd,Vine - Make a Scene,floofer
3447,None,Just wanted to share this super rare Rainbow Floofer in case you guys haven't seen it yet. 13/10 colorful af,737445876994609152,13,10,https://t.co/CaG9MzD3WT,Twitter for iPhone,floofer
3466,Moose,This is Moose. He's a Polynesian Floofer. Dapper af. 10/10 would pet diligently,733822306246479872,10,10,https://t.co/mVfqRdppTL,Twitter for iPhone,floofer
3890,None,Here we are witnessing a rare High Stepping Alaskan Floofer. 12/10 dangerously petable (vid by @TheMrsNux),689993469801164801,12,10,https://t.co/K4s9IJh2jm,Vine - Make a Scene,floofer
3970,Petrick,Say hello to Petrick. He's an Altostratus Floofer. Just had a run in with a trash bag. Groovy checkered floor. 11/10,685307451701334016,11,10,https://t.co/rwW7z1JAOF,Twitter for iPhone,floofer


### 2. Images from Twitter

In [24]:
#create a folder
folder = "image_predictions"
if not os.path.exists(folder):
    os.makedirs(folder)

#Using the URL to get the images from Twitter 
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

#downloading the files to the specified folder
response = requests.get(url)
with open(os.path.join(folder + "/image-predictions.tsv"), "wb") as file: #wb means writing in binary mode
    file.write(response.content)

In [25]:
images = pd.read_csv("image_predictions//image-predictions.tsv", sep = "\t") #seperate the strings by tabular 
images.head()

#p1,2, and 3 are the three top dogs the algorithm has predicted that the breed of the dog will be the image in the Tweet.
#p1_conf, p2_conf, and p3_conf are how confident the algorithm is in predicting the iamge is correct
#p1_dog, p2_dog, and p3_dog idicates whether or not the breed of the dog was predicted correctly. 

#check that the images are actually dogs. box_turtle, three-toed_sloth and hen are not
#check that the names are changed to the actual dog breeds. shopping_cart is actually a golden retriever, desktop_computer has a dog too. 
#the urls without the jpg at the end are not working. - how can I get the correct URL?

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [26]:
images.iloc[2052]

tweet_id    887517139158093824                                                                     
jpg_url     https://pbs.twimg.com/ext_tw_video_thumb/887517108413886465/pu/img/WanJKwssZj4VJvL9.jpg
img_num     1                                                                                      
p1          limousine                                                                              
p1_conf     0.130432                                                                               
p1_dog      False                                                                                  
p2          tow_truck                                                                              
p2_conf     0.0291754                                                                              
p2_dog      False                                                                                  
p3          shopping_cart                                                                          


In [27]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [28]:
images["img_num"].value_counts()

1    1780
2    198 
3    66  
4    31  
Name: img_num, dtype: int64

In [29]:
#make a new copy of the dataframe so the original dataframe is intact. 
images_clean = images.copy()
images_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [30]:
#make sure that the predictions are acutally dogs
breed = []
confidence = []

def breed_confidence(row):
    if row['p1_dog'] == True:
        breed.append(row['p1'])
        confidence.append(row['p1_conf'])
    elif row['p2_dog'] == True:
        breed.append(row['p2'])
        confidence.append(row['p2_conf'])
    elif row['p3_dog'] == True:
        breed.append(row['p3'])
        confidence.append(row['p3_conf'])
    else:
        breed.append('Not a dog')
        confidence.append(0)
        
images_clean.apply(breed_confidence, axis = 1)
images_clean['breed'] = breed
images_clean['confidence'] = confidence
images_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,breed,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True,German_shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,Rhodesian_ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True,miniature_pinscher,0.560311


In [31]:
#drop columns that are not needed
images_clean.drop(['img_num','p1','p1_conf','p1_dog','p2','p2_dog','p2_conf','p3','p3_dog','p3_conf'], axis = 1, inplace = True)
images_clean

,tweet_id,jpg_url,breed,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German_shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,Rhodesian_ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,miniature_pinscher,0.560311
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,Bernese_mountain_dog,0.651137
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,Not a dog,0.000000
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,chow,0.692517
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,golden_retriever,0.007959
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,miniature_poodle,0.201493


In [32]:
#delete the rows that are not dogs
images_clean.drop(images_clean[images_clean['breed'] == "Not a dog"].index, inplace = True)
images_clean.head()
images_clean.query('breed == "Not a dog"').count()

tweet_id      0
jpg_url       0
breed         0
confidence    0
dtype: int64

In [33]:
#Some url's do not work because the column has been truncated in the dataframe
pd.set_option('display.max_colwidth', -1)
images_clean.head()

,tweet_id,jpg_url,breed,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German_shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,Rhodesian_ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,miniature_pinscher,0.560311


In [34]:
#delete "_" in between the words and capitalize each word to keep it consistent 
images_clean["breed"] = images_clean.breed.str.replace("_"," ").str.title()

In [35]:
#rename column, so that we can compare both jpg urls from the Twitter archive and the image prediction
images_clean = images_clean.rename(columns={'jpg_url': 'predicted_jpg_url'})
images_clean

,tweet_id,predicted_jpg_url,breed,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh Springer Spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,Redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German Shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,Rhodesian Ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,Miniature Pinscher,0.560311
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,Bernese Mountain Dog,0.651137
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,Chow,0.692517
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,Golden Retriever,0.007959
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,Miniature Poodle,0.201493
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,Golden Retriever,0.775930


In [36]:
images_clean.breed.value_counts()

Golden Retriever                  173
Labrador Retriever                113
Pembroke                          96 
Chihuahua                         95 
Pug                               65 
Toy Poodle                        52 
Chow                              51 
Samoyed                           46 
Pomeranian                        42 
Malamute                          34 
Cocker Spaniel                    34 
French Bulldog                    32 
Chesapeake Bay Retriever          31 
Miniature Pinscher                26 
Cardigan                          23 
Eskimo Dog                        22 
Staffordshire Bullterrier         22 
German Shepherd                   21 
Beagle                            21 
Shih-Tzu                          20 
Siberian Husky                    20 
Rottweiler                        19 
Shetland Sheepdog                 19 
Kuvasz                            19 
Lakeland Terrier                  19 
Maltese Dog                       19 
Italian Grey

### 3. Data from Twitter API (Retweet/Favourite Count)

In [37]:
consumer_key = 'UDFvUBJONqeZAN7CiirOHqlnf'
consumer_secret = 'X0HlRpj4ncfKULTRntf83uD4IFKUo1EGMz4X6zQdLhCP2sJjew'
access_token = '1033698097598214148-1PItGIRDQYBcDdeuberGDQYqLCUdhj'
access_secret = '8ycsLG30C40I1tXa5e7deBEEtCdfNrPwrDqmn1TOk4O9N'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [38]:
tweet_ids = arc_clean.tweet_id.values
len(tweet_ids)

2453

In [39]:
#tweepy library, store each tweet in tweet_json.txt

tweets_list =[]
with open('tweet_json.txt') as json_file:
    for line in json_file:
    
        tweets_dict = {}
        tweets_json = json.loads(line)
        
        tweets_dict['tweet_id'] = tweets_json['id']
        tweets_dict['retweet_count'] = tweets_json['retweet_count']
        tweets_dict['favorite_count'] = tweets_json['favorite_count']
        
        tweets_list.append(tweets_dict)

In [40]:
#convert the text into a dataframe 

tweets_stats = pd.DataFrame(tweets_list)
tweets_stats.head()

,favorite_count,retweet_count,tweet_id
0,39467,8853,892420643555336193
1,33819,6514,892177421306343426
2,25461,4328,891815181378084864
3,42908,8964,891689557279858688
4,41048,9774,891327558926688256


In [41]:
tweets_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


### Merge Dataframes

In [42]:
arc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2453 entries, 0 to 1204
Data columns (total 8 columns):
name                  2453 non-null object
texts                 2453 non-null object
tweet_id              2453 non-null int64
rating_numerator      2453 non-null int64
rating_denominator    2453 non-null int64
url                   2452 non-null object
sourced               2453 non-null object
dog_stage             2453 non-null object
dtypes: int64(3), object(5)
memory usage: 252.5+ KB


In [43]:
images_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 0 to 2073
Data columns (total 4 columns):
tweet_id             1751 non-null int64
predicted_jpg_url    1751 non-null object
breed                1751 non-null object
confidence           1751 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 68.4+ KB


In [44]:
tweets_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [45]:
#merge arc_clean, images_clean and tweets_stats
#column names that are the same: tweet_id
master_df_1 = pd.merge(arc_clean, images_clean, on='tweet_id', how='left')
master_df = pd.merge(master_df_1, tweets_stats, on="tweet_id", how = "left")
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2453 entries, 0 to 2452
Data columns (total 13 columns):
name                  2453 non-null object
texts                 2453 non-null object
tweet_id              2453 non-null int64
rating_numerator      2453 non-null int64
rating_denominator    2453 non-null int64
url                   2452 non-null object
sourced               2453 non-null object
dog_stage             2453 non-null object
predicted_jpg_url     1820 non-null object
breed                 1820 non-null object
confidence            1820 non-null float64
favorite_count        2451 non-null float64
retweet_count         2451 non-null float64
dtypes: float64(3), int64(3), object(7)
memory usage: 268.3+ KB


#### Assess the Data

In [48]:
#what is maximum numerator rating given to the dog?
master_df["rating_numerator"].max()

1776

In [49]:
master_df["rating_numerator"].min()

0

In [50]:
#what is the most popular dog name?
master_df["name"].value_counts()

#There are 931 names recorded and Charlie is the most popular dog name

None         902
Charlie      12 
Bo           11 
Cooper       11 
Oliver       11 
Lucy         11 
Tucker       10 
Penny        10 
Lola         10 
Winston      9  
Sadie        8  
Daisy        7  
Bailey       7  
Buddy        7  
Scout        7  
Toby         7  
Sunny        7  
Jack         6  
Milo         6  
Loki         6  
Koda         6  
Sampson      6  
Jax          6  
Rusty        6  
Leo          6  
Gerald       6  
Dave         6  
Stanley      6  
Bella        6  
Oscar        6  
            ..  
Freddery     1  
Mark         1  
Pherb        1  
Beckham      1  
Kane         1  
Lilah        1  
Lorelei      1  
Reptar       1  
Cilantro     1  
Walker       1  
Ivar         1  
Danny        1  
Kulet        1  
Stormy       1  
Ronduh       1  
Travis       1  
Ralphie      1  
Stewie       1  
Maya         1  
Jangle       1  
Raphael      1  
Cheryl       1  
Poppy        1  
Dylan        1  
Clifford     1  
Severus      1  
Tug          1  
Rodney       1

In [51]:
#most popular predicted breed
master_df["breed"].value_counts()

#Out of 113 breeds recorded, golden retriever was predicted to 190 times

Golden Retriever              190
Labrador Retriever            119
Pembroke                      98 
Chihuahua                     96 
Pug                           65 
Toy Poodle                    52 
Chow                          51 
Samoyed                       46 
Pomeranian                    44 
Cocker Spaniel                36 
Malamute                      34 
French Bulldog                32 
Chesapeake Bay Retriever      32 
Miniature Pinscher            26 
Staffordshire Bullterrier     25 
German Shepherd               24 
Cardigan                      23 
Eskimo Dog                    23 
Beagle                        22 
Siberian Husky                21 
Shih-Tzu                      20 
Maltese Dog                   19 
Rottweiler                    19 
Basset                        19 
Kuvasz                        19 
Shetland Sheepdog             19 
Lakeland Terrier              19 
Italian Greyhound             17 
Great Pyrenees                17 
Old English Sh

In [52]:
#most retweeted dog post
master_df["retweet_count"].max()

79515.0

In [53]:
#most favourited dog post
master_df["favorite_count"].max()

132810.0

#### Store the Data

In [ ]:
# Saving data to csv

folder = 'WeRateDogs_Project'
if not os.path.exists(folder):
    os.makedirs(folder)
    
    
master_df.to_csv('WeRateDogs_Project/twitter_archive_master.csv')
arc_clean.to_csv('WeRateDogs_Project/clean_twitter_archive_data.csv')
images_clean.to_csv('WeRateDogs_Project/clean_image_prediction_data.csv')
tweets_stats.to_csv('WeRateDogs_Project/tweets_stats.csv')